In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from scipy.sparse import hstack


In [2]:
df = pd.read_csv("data\cleaned_output_file.csv")



In [3]:
#min max scaler for rating and year
scaler = MinMaxScaler()
df['Rating Scale'] = scaler.fit_transform(df[['Average Rating']])
df['Year Scale'] = scaler.fit_transform(df[['Publish Date']])



#tf idf for synop + reviews
vectorizer = TfidfVectorizer(ngram_range=(1,2))

columnstoVectorize = ("Proccessed Synopsis" , "Proccessed Review")

reviews_tfidf = vectorizer.fit_transform(df['Proccessed Review'])
synopsis_tfidf = vectorizer.fit_transform(df['Proccessed Synopsis'])

#genres, book titles authors
if df['Genres'] is str:
    df['Genres'] = df['Genres'].str.split("'")
    print(df['Genres'])
encoder = OneHotEncoder(handle_unknown='ignore')
encoded_genres = encoder.fit_transform(df['Genres'].apply(lambda x: pd.Series(x)).stack().reset_index(level=1, drop=True).to_frame('Genres'))
encoded_titles = encoder.fit_transform(df[['Book Title']])
encoded_authors = encoder.fit_transform(df[['Author']])
encoded_features = hstack([encoded_titles, encoded_authors, encoded_genres])

#combing everythng together
combined_features = hstack([encoded_features, df[['Rating Scale', 'Year Scale']], reviews_tfidf, synopsis_tfidf])


In [4]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel
import numpy as np

similarity_matrix = cosine_similarity(combined_features)



In [5]:
def get_recommendations(title, similarity_matrix, df, top_n=10):
    idx = df.index.get_loc(title)
    sim_scores = list(enumerate(similarity_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_n+1]
    book_indices = [i[0] for i in sim_scores]
    
    print(df.iloc[book_indices]["Synopsis"])
    return df.iloc[book_indices].index.tolist()




In [33]:
if 1 in df.index:
    df.set_index('Book Title', inplace=True)
else:
    pass


import tkinter as tk
from tkinter import ttk

class AutocompleteCombobox(ttk.Combobox):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.config(font=("Times New Roman", 14))
        self.book_titles = sorted(kwargs.get('values', []))  # List of book titles
        self._completion_list = self.book_titles
        self.bind('<Enter>', self.handle_keyrelease)
        self._hits = []

    def handle_keyrelease(self, event):
        if event.keysym == "BackSpace":
            self.delete(self.index(tk.INSERT), tk.END)
        elif event.keysym == "Left" or event.keysym == "Right":
            return
        elif event.keysym == "Return":  # Handle Enter key
            self._update_autocomplete()
            self.event_generate('<Down>')
        else:
            self._update_autocomplete()

    def _update_autocomplete(self):
        typed = self.get()
        if typed == '':
            self._hits = self._completion_list
        else:
            self._hits = [title for title in self._completion_list if typed.lower() in title.lower()]
        if self._hits:
            self['values'] = self._hits
            self.event_generate('<Down>')


def update_recommendations(event):
    selected_book = book_var.get()
    recommendations = get_recommendations(selected_book, similarity_matrix, df)
    recommendation_list.delete(0, tk.END)
    for i, book in enumerate(recommendations):
        synopsis = df.loc[book]["Synopsis"]
        recommendation_list.insert(tk.END, f"{i+1}.{book}")
        recommendation_list.insert(tk.END, f"Synopsis: {synopsis[:50]}...")

# Tkinter window setup
root = tk.Tk()
root.title("Book Recommendation System")
root.geometry("800x600")
              
book_var = tk.StringVar()
book_titles = df.index.tolist()

book_dropdown = AutocompleteCombobox(root, textvariable=book_var, values=book_titles)
book_dropdown.grid(row=0, column=0, padx=10, pady=10)
book_dropdown.bind('<<ComboboxSelected>>', update_recommendations)

recommendation_list = tk.Listbox(root, width=130, height=33)
recommendation_list.grid(row=1, column=0, padx=10, pady=10)

root.mainloop()

Book Title
A Walk in the Woods: Rediscovering America on the Appalachian Trail (Mass Market Paperback)    appalachian trail stretch georgia maine cover ...
Sapiens: A Brief History of Humankind (Paperback)                                              renowned historian come groundbreaking narrati...
Educated (Hardcover)                                                                           tara westover 17 first time set foot classroom...
Becoming (Hardcover)                                                                           life filled meaning accomplishment michelle ob...
When Breath Becomes Air (Kindle Edition)                                                       reader atul gawande andrew solomon anne lamott...
I'm Glad My Mom Died (ebook)                                                                   heartbreaking hilarious memoir icarly sam cat ...
Between the World and Me (Hardcover)                                                           country world body must 

In [11]:
if 1 in df.index:
    df.set_index('Book Title', inplace=True)
else:
    pass

import tkinter as tk
from tkinter import ttk,simpledialog
class AutocompleteCombobox(ttk.Combobox):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.config(font=("Times New Roman", 14))
        self.book_titles = sorted(kwargs.get('values', []))  # List of book titles
        self._completion_list = self.book_titles
        self.bind('<Enter>', self.handle_keyrelease)
        self._hits = []

    def handle_keyrelease(self, event):
        if event.keysym == "BackSpace":
            self.delete(self.index(tk.INSERT), tk.END)
        elif event.keysym == "Left" or event.keysym == "Right":
            return
        elif event.keysym == "Return":  # Handle Enter key
            self._update_autocomplete()
            self.event_generate('<Down>')
        else:
            self._update_autocomplete()

    def _update_autocomplete(self):
        typed = self.get()
        if typed == '':
            self._hits = self._completion_list
        else:
            self._hits = [title for title in self._completion_list if typed.lower() in title.lower()]
        if self._hits:
            self['values'] = self._hits
            self.event_generate('<Down>')


def update_recommendations(event):
    selected_book = book_var.get()
    recommendations = get_recommendations(selected_book, similarity_matrix, df)
    recommendation_list.delete(0, tk.END)
    for i, book in enumerate(recommendations):
        synopsis = df.loc[book]["Synopsis"]
        recommendation_list.insert(tk.END, f"{i+1}. {book} - Synopsis: {synopsis[:50]}...")
        
def show_synopsis(event):
    selected_index = recommendation_list.curselection()
    if selected_index:
        selected_book = recommendation_list.get(selected_index[0]).split(" - ")[0][3:]
        synopsis = df.loc[selected_book, "Synopsis"]
        simpledialog.Dialog(title=selected_book, text=synopsis, master=root).show()

# Tkinter window setup
root = tk.Tk()
root.title("Book Recommendation System")
root.geometry("800x600")

book_var = tk.StringVar()
book_titles = df.index.tolist()


book_dropdown = AutocompleteCombobox(root, textvariable=book_var, values=book_titles)
book_dropdown.grid(row=0, column=0, padx=10, pady=10)
book_dropdown.bind('<<ComboboxSelected>>', update_recommendations)

recommendation_list = tk.Listbox(root, width=130, height=33, font=("Times New Roman", 12))
recommendation_list.grid(row=1, column=0, padx=10, pady=10)
recommendation_list.bind('<Double-1>', show_synopsis) 

root.mainloop()

Book Title
We Should All Be Feminists (Kindle Edition)                           feminism mean today question heart feminist pe...
I'm Glad My Mom Died (ebook)                                          heartbreaking hilarious memoir icarly sam cat ...
Educated (Hardcover)                                                  tara westover 17 first time set foot classroom...
Becoming (Hardcover)                                                  life filled meaning accomplishment michelle ob...
Sapiens: A Brief History of Humankind (Paperback)                     renowned historian come groundbreaking narrati...
When Breath Becomes Air (Kindle Edition)                              reader atul gawande andrew solomon anne lamott...
Invisible Women: Data Bias in a World Designed for Men (Hardcover)    data fundamental modern world economic develop...
Born a Crime: Stories From a South African Childhood (Hardcover)      memoir one man coming age set twilight aparthe...
Between the World and Me (Har

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\ieyua\anaconda3\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\ieyua\AppData\Local\Temp\ipykernel_3900\1188915966.py", line 52, in show_synopsis
    simpledialog.Dialog(title=selected_book, text=synopsis, master=root).show()
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: Dialog.__init__() got an unexpected keyword argument 'text'
Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\ieyua\anaconda3\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\ieyua\AppData\Local\Temp\ipykernel_3900\1188915966.py", line 52, in show_synopsis
    simpledialog.Dialog(title=selected_book, text=synopsis, master=root).show()
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: Dialog._